**Basic Steps**
1. Import necessary packages
2. Create fully connected networks
3. Set devices
4. Hyperparameters
5. Load data
6. Loss and optimiser
7. Train network
8. Check accuracy on training & test to see how good our model

In [3]:
import torch
import torch.nn as nn # neural network and loss function
import torch.optim as optim # optimisation algorithm
import torch.nn.functional as F
from torch.utils.data import DataLoader # easier to data management
import torchvision.datasets as datasets # standard datasets
import torchvision.transforms as transforms # transformations we can perform on the dataset

In [5]:
# Create fully connected networks
'''
fully connected means every neuron in one layer connects to every neuron in the next layer. Each connection has its own weight, and each neuron usually has a bias term.
'''
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [6]:
# set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
# Hyperparameters
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 1 # one round of forward and backward propagation

In [8]:
# Load data
train_dataset = datasets.MNIST(root = 'dataset/', train = True, download = True, transform = transforms.ToTensor())
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_dataset = datasets.MNIST(root = 'dataset/', train = False, download = True, transform = transforms.ToTensor())
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

100.0%
100.0%
100.0%
100.0%


In [9]:
# initialise neural network
model = NN(input_size, num_classes).to(device)

In [11]:
# Loss and optimiser
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [15]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # get data to cuda if possible
        data = data.to(device = device)
        targets = targets.to(device = device)
        # get to correct shape
        data = data.reshape(data.shape[0], -1)
        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        # backward
        optimizer.zero_grad() # set all gradients to zero for each batch
        loss.backward()

        #gradient descent or adam step
        optimizer.step()

In [21]:
# check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training dataset")
    else:
        print("Checking accuracy on test dataset")
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for data, target in loader:
            data = data.to(device = device)
            target = target.to(device = device)
            data = data.reshape(data.shape[0], -1)
            scores = model(data)
            _, predictions = scores.max(1)
            num_correct += (predictions == target).sum()
            num_samples += predictions.size(0)
        print(f"Got {num_correct} / {num_samples} correct predictions with accuracy {num_correct / num_samples * 100:.2f}%")
        acc = num_correct / num_samples * 100
        model.train()
        return acc

In [22]:
check_accuracy(train_loader, model)

Checking accuracy on training dataset
Got 55675 / 60000 correct predictions with accuracy 92.79%


tensor(92.7917, device='cuda:0')

In [23]:
check_accuracy(test_loader, model)

Checking accuracy on test dataset
Got 9294 / 10000 correct predictions with accuracy 92.94%


tensor(92.9400, device='cuda:0')